In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BATCH_SIZE = 8
max_length=1024
PATH_TO_FOLDER = f'/content/drive/MyDrive/CommonLit/deberta-v3-base/batch_{BATCH_SIZE}/{max_length}'

In [ ]:
import pandas as pd
from tqdm import tqdm
import torch
import numpy as np
import torch.nn as nn
import gc
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,GroupKFold

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c commonlit-evaluate-student-summaries

commonlit-evaluate-student-summaries.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
with zipfile.ZipFile("/content/commonlit-evaluate-student-summaries.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
data = pd.read_csv('/content/summaries_train.csv')

X = data['text'].to_list()
y = data[['wording', 'content']].values

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/CommonLit/nlp_features_csv.csv")
data.drop([7165,7166,7167,7168],0,inplace=True)
selected_vars = ['content', 'wording', 'word_count', 'sentence_length', 'vocabulary_richness', 'avg_word_length',
                   'comma_count', 'semicolon_count', 'exclamation_count', 'question_count',
                   'quote_count', 'unique_word_count', 'pos_mean', 'compound', 'pos', 'neg', 'neu',
                   'punctuation_sum', 'word_overlap', 'prompt_length', 'text_to_prompt_ratio', 'keyword_density'
                  ,'compound_prompt', 'pos_prompt', 'neg_prompt', 'neu_prompt', 'jaccard_similarity']
train = data[selected_vars]

X = train.drop(['wording','content'],1)
X = X.to_numpy()
y = train[['wording', 'content']].values

<ipython-input-19-4243817690ce>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop([7165,7166,7167,7168],0,inplace=True)
<ipython-input-19-4243817690ce>:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = train.drop(['wording','content'],1)


In [ ]:
group_kfold = GroupKFold(n_splits=4)
folds = {'39c16e':0,
         '3b9047':1,
         'ebad26':2,
         '814d6b':3
}
groups = np.array([folds[data['prompt_id'][i]] for i in range(len(data))])
dfs = []
for i, (train_index, test_index) in tqdm(enumerate(group_kfold.split(X, y, groups))):
    print(f'Begining Fold:{i+1}')

    model_wording = lgb.LGBMRegressor(random_state=42)
    model_content = lgb.LGBMRegressor(random_state=42)

    model_wording.fit(X[train_index],y[train_index][:,0])
    word_prediction = model_wording.predict(X[test_index])


    model_content.fit(X[train_index],y[train_index][:,1])
    content_prediction = model_content.predict(X[test_index])

    df = pd.DataFrame(index=test_index)
    df['true_word'] = y[test_index][:,0]
    df['true_content'] = y[test_index][:,1]
    df['pred_word'] = word_prediction
    df['pred_content'] = content_prediction
    dfs.append(df)

0it [00:00, ?it/s]

Begining Fold:1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000882 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3013
[LightGBM] [Info] Number of data points in the train set: 5108, number of used features: 25
[LightGBM] [Info] Start training from score -0.031791


1it [00:00,  2.01it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000615 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3013
[LightGBM] [Info] Number of data points in the train set: 5108, number of used features: 25
[LightGBM] [Info] Start training from score 0.017606
Begining Fold:2
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001521 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3050
[LightGBM] [Info] Number of data points in the train set: 5156, number of used features: 24
[LightGBM] [Info] Start training from score -0.060941
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001466 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3050
[LightGBM] [Info] Number of data

2it [00:00,  2.05it/s]

Begining Fold:3
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002487 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3085
[LightGBM] [Info] Number of data points in the train set: 5169, number of used features: 25
[LightGBM] [Info] Start training from score 0.028040


3it [00:01,  2.11it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000606 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3085
[LightGBM] [Info] Number of data points in the train set: 5169, number of used features: 25
[LightGBM] [Info] Start training from score 0.013356
Begining Fold:4
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001624 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3117
[LightGBM] [Info] Number of data points in the train set: 6062, number of used features: 24
[LightGBM] [Info] Start training from score -0.168933


4it [00:01,  2.09it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3117
[LightGBM] [Info] Number of data points in the train set: 6062, number of used features: 24
[LightGBM] [Info] Start training from score -0.044904


In [ ]:
group_kfold = GroupKFold(n_splits=4)
folds = {'39c16e':0,
         '3b9047':1,
         'ebad26':2,
         '814d6b':3
}
groups = np.array([folds[data['prompt_id'][i]] for i in range(len(data))])
dfs1 = []
for i, (train_index, test_index) in tqdm(enumerate(group_kfold.split(X, y, groups))):
    print(f'Begining Fold:{i+1}')

    model_wording = GradientBoostingRegressor(random_state=42)
    model_content = GradientBoostingRegressor(random_state=42)

    model_wording.fit(X[train_index],y[train_index][:,0])
    word_prediction = model_wording.predict(X[test_index])


    model_content.fit(X[train_index],y[train_index][:,1])
    content_prediction = model_content.predict(X[test_index])

    df = pd.DataFrame(index=test_index)
    df['true_word'] = y[test_index][:,0]
    df['true_content'] = y[test_index][:,1]
    df['pred_word'] = word_prediction
    df['pred_content'] = content_prediction
    dfs1.append(df)

0it [00:00, ?it/s]

Begining Fold:1


1it [00:03,  3.72s/it]

Begining Fold:2


2it [00:08,  4.44s/it]

Begining Fold:3


3it [00:12,  4.11s/it]

Begining Fold:4


4it [00:16,  4.18s/it]


In [ ]:
total_w = 0.0
total_c = 0.0
total_mcrmse = 0.0
i = 0
for df,df1 in (zip(dfs,dfs1)):
    df_deberta = pd.read_csv(PATH_TO_FOLDER+f'/predictions_fold_{i+1}.csv',index_col="idx")
    word_pred = (df['pred_word'] + df_deberta['pred_word'] + df1['pred_word']) / 3
    content_pred = (df['pred_content'] + df_deberta['pred_content'] + df1['pred_content']) / 3
    rmse_w = mean_squared_error(df_deberta['true_word'],word_pred,squared=False)
    rmse_c = mean_squared_error(df_deberta['true_content'],content_pred,squared=False)
    print(f"Fold {i+1}")
    print(f"Wording:{rmse_w}")
    print(f"Content:{rmse_c}")
    print(f"MCRMSE:{(rmse_w+rmse_c)/2}")

    total_w+=rmse_w
    total_c+=rmse_c
    total_mcrmse+=(rmse_w+rmse_c)/2
    i+=1
print("Total")
print(f"Wording:{total_w/4}")
print(f"Content:{total_c/4}")
print(f"MCRMSE:{total_mcrmse/4}")

Fold 1
Wording:0.5275593813358224
Content:0.4612529823459557
MCRMSE:0.494406181840889
Fold 2
Wording:0.8200424149352418
Content:0.5023597823952067
MCRMSE:0.6612010986652243
Fold 3
Wording:0.5037425587269663
Content:0.42972597915880495
MCRMSE:0.4667342689428856
Fold 4
Wording:0.7570528054633436
Content:0.611388124796624
MCRMSE:0.6842204651299838
Total
Wording:0.6520992901153435
Content:0.5011817171741478
MCRMSE:0.5766405036447456
